In [ ]:
import sqlite3
from IPython.display import HTML, display

db = 'file:results.db?mode=ro'
con = sqlite3.connect(db, uri=True)

cur = con.cursor()
cur.execute("SELECT * FROM runs WHERE description LIKE '%scaling%'")

def display_result(cur, data=cur.fetchall()):
    html = "<table>"
    headers = list(map(lambda x: x[0], cur.description))
    
    html += "<thead><tr>" + "".join(["<td>" + h + "</td>" for h in headers]) + "</tr></thead>"
    for row in data:
        html += "<tr>"
        for field in row:
            html += "<td>%s</td>"%(field)
        html += "</tr>"
    html += "</table>"
    display(HTML(html))
display_result(cur)

In [ ]:
benchmark_id = 23

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
from matplotlib.ticker import EngFormatter

f = plt.figure()
ax = f.subplots(1)
ax.set_xlabel("Number of ranks")
ax.set_ylabel("Speedup")

cur.execute("SELECT description FROM runs WHERE id = ?", (benchmark_id,))
description = cur.fetchone()[0]

cur.execute("SELECT time_ns, n_summands FROM results WHERE run_id=? AND ranks = 1", (benchmark_id,))
r = cur.fetchone()
sequential_time = r[0] * 1e-9

ax.set_title(f"{description} of {r[1]} summands")

cur.execute("SELECT ranks, time_ns FROM results WHERE run_id = ? AND ranks > 1 ORDER BY ranks", [benchmark_id])

X = [1] # core count
Y = [1] # speed-up

for row in cur.fetchall():
    X.append(row[0])
    parallel_time = row[1] * 1e-9
    speedup = sequential_time / parallel_time
    Y.append(speedup)

plt.scatter(X, Y)
plt.plot(X, Y)
plt.show()